# Overview

# Imports

In [ ]:
import pandas as pd
import sqlalchemy as sa

In [ ]:
%run sql_magic.ipynb

# Configure database

In [ ]:
DB_URL = "postgres://postgres:@localhost:5432/postgres"

## Set up `University` schema

Only do this if you are setting up PostgreSQL for the first time!

In [ ]:
!psql {DB_URL} -f ../data/coursesDDL.txt >/dev/null

In [ ]:
!psql {DB_URL} -f ../data/coursesData.txt >/dev/null

## Create database engine

In [ ]:
engine = sa.create_engine(DB_URL, connect_args={'options': '-csearch_path=University'})

In [ ]:
engine.table_names()

# Exercises

## SQL: Outer Joins etc.

#### 1. Which of these queries is legal?

In [ ]:
%%sql
SELECT count(distinct dept), count(distinct instructor)
FROM Offering
WHERE term >= 20089;

In [ ]:
%%sql
SELECT distinct dept, distinct instructor
FROM Offering
WHERE term >= 20089;

In [ ]:
%%sql
SELECT distinct dept, instructor
FROM Offering
WHERE term >= 20089;

#### 2. Under what conditions could these two queries give different results? If that is not possible, explain why.

In [ ]:
%%sql
SELECT surName, campus
FROM Student;

In [ ]:
%%sql
SELECT distinct surName, campus
FROM Student;

#### 3. For each student who has taken a course, report their sid and the number of different departments they have taken a course in.

In [ ]:
%%sql
SELECT sid, count(distinct dept)
FROM Took JOIN Offering ON Took.oid = Offering.oid
GROUP BY sid;

#### 4. Suppose we have two tables with content as follows:

In [ ]:
%%sql # One
drop table if exists One;

create table One (
    a int,
    b int
);

insert into One values (1, 2), (6, 12), (null, 100), (20, null);

select * from One;

In [ ]:
%%sql # Two
drop table if exists Two;

create table Two (
    b int,
    c int
);

insert into Two values (2, 3), (100, 101), (20, 21), (2, 4), (2, 5);

select * from Two;

(a) What query could produce this result?

| a | b | c |
|- | - | - |
| 1 | 2 | 3 |
| 1 | 2 | 4 |
| 1 | 2 | 5 |
| | 20 | 21 |
| | 100 | 101 |

In [ ]:
%%sql
SELECT * FROM Two NATURAL LEFT JOIN One;

In [ ]:
%%sql
SELECT * FROM One NATURAL RIGHT JOIN Two;

(b) What query could produce this result?

| a | b | c |
| - | - | - |
| 1 | 2 | 3 |
| 1 | 2 | 4 |
| 1 | 2 | 5 |
| 6 | 12 | |
| | 100 | 101 |
| 20 | | |

In [ ]:
%%sql
SELECT * FROM One NATURAL LEFT JOIN Two;

In [ ]:
%%sql
SELECT * FROM Two NATURAL RIGHT JOIN One;

## Subqueries: Solutions

#### 1. What does this query do? (Recall that the || operator concatenates two strings.)

In [ ]:
%%sql
SELECT sid, dept||cnum as course, grade
FROM Took,
(
    SELECT *
    FROM Offering
    WHERE instructor = 'Horton'
) Hoffering
WHERE Took.oid = Hoffering.oid;

#### 2. What does this query do?

In [ ]:
%%sql
SELECT sid, surname
FROM Student
WHERE cgpa >
(
    SELECT cgpa
    FROM Student
    WHERE sid = 99999  -- 11111
);

#### 3. What does this query do?

In [ ]:
%%sql
SELECT sid, dept||cnum AS course, grade
FROM Took NATURAL JOIN Offering
WHERE grade >= 80 AND
(cnum, dept) IN (
    SELECT cnum, dept
    FROM Took NATURAL JOIN Offering NATURAL JOIN Student
    WHERE surname = 'Lakemeyer'
);

#### 4. Suppose we have these relations: R(a, b) and S(b, c).

a) What does this query do?

```sql
SELECT a
FROM R
WHERE b in (SELECT b FROM S);
```

b) Can we express this query without using subqueries?

```sql
SELECT a
FROM R NATURAL JOIN S
```

#### 5. What does this query do?

In [ ]:
%%sql
SELECT instructor
FROM Offering Off1
WHERE NOT EXISTS (
    SELECT *
    FROM Offering
    WHERE oid <> Off1.oid
    AND instructor = Off1.instructor
);

#### 6. What does this query do?

In [ ]:
%%sql
SELECT DISTINCT oid
FROM Took
WHERE EXISTS (
    SELECT *
    FROM Took t, Offering o
    WHERE t.oid = o.oid
    AND t.oid <> Took.oid
    AND o.dept = 'CSC'
    AND took.sid = t.sid
);

#### 7. Now let’s write some queries!

For each course, that is, each department and course number combination, find the instructor who has taught the most offerings of it. If there are ties, include them all. Report the course (eg "csc343"), instructor and the number of offerings of the course by that instructor.

(a) First, create a view called Counts to hold, for each course, and each instructor who has taught it, their
number of offerings.

In [ ]:
%%sql
-- This intermediate result is helpful:
CREATE OR REPLACE VIEW Counts as
SELECT dept || cnum as course, instructor, count(oid)
FROM Offering
GROUP BY cnum, dept, instructor;

COMMIT;

-- Let’s take a look at what this computes.
-- (Our dataset doesn’t give this view a very good test.)
SELECT * from Counts;

(b) Now solve the problem. Do not use any joins. (This will force you to use a subquery.)

In [ ]:
%%sql
-- Now we can solve the problem using a subquery:
SELECT course, instructor, count
FROM Counts C1
WHERE count >= ALL (
    SELECT count
    FROM Counts C2
    WHERE C1.course = C2.course)
ORDER BY C1.course;

In [ ]:
%%sql
-- Here’s another version:
SELECT course, instructor, count
FROM Counts C1
WHERE count = (
    SELECT max(count)
    FROM Counts C2
    WHERE C1.course = C2.course)
ORDER BY C1.course;

#### 8. Use EXISTS to find the surname and email address of students who have never taken a CSC course.

In [ ]:
%%sql
SELECT surname, email
FROM Student
WHERE NOT EXISTS (
SELECT *
FROM Took JOIN Offering ON Took.oid = Offering.oid
WHERE Took.sid = Student.sid AND Offering.dept = 'CSC');

#### 9. Use EXISTS to find every instructor who has given a grade of 100.

In [ ]:
%%sql
SELECT distinct instructor
FROM Offering O1
WHERE EXISTS (
    SELECT *
    FROM Offering O2 JOIN Took ON O2.oid = Took.oid
    WHERE O1.instructor = O2.instructor AND grade = 100);

#### 10. Let’s say that a course has level "junior" if its cNum is between 100 and 299 inclusive, and has level "senior" if its cNum is between 300 and 499 inclusive.

Report the average grade, across all departments and course offerings, for all junior courses and for all senior courses. Report your answer in a table that looks like this:

| level | levelavg |
| ---------|----------- |
| junior | |
| senior | |

Each average should be an average of the individual student grades, not an average of the course averages.

In [ ]:
%%sql
CREATE OR REPLACE VIEW Grades AS
SELECT cnum, dept, grade
FROM Offering natural join Took;

(SELECT 'junior' AS level, avg(grade) AS levelavg
 FROM Grades
 WHERE cnum >= 100 AND cnum <= 299)
UNION ALL
(SELECT 'senior' AS level, avg(grade) AS levelavg
 FROM Grades
 WHERE cnum >= 300 AND cnum <= 499);